In [1]:
import pandas as pd
import string

# Verisetinin Okunması

In [21]:
halk_dataset = pd.read_csv("../raw_data/halk.csv")
sanat_dataset = pd.read_csv("../raw_data/sanat.csv")
ilahi_dataset = pd.read_csv("../raw_data/ilahi.csv")
pop_dataset = pd.read_csv("../raw_data/pop.csv")
rock_dataset = pd.read_csv("../raw_data/rock.csv")
rap_dataset = pd.read_csv("../raw_data/rap.csv")

# Veri Etiketleme
halk_dataset["sarki_turu"] = ["halk" for _ in range(200)]
sanat_dataset["sarki_turu"] = ["sanat" for _ in range(200)]
ilahi_dataset["sarki_turu"] = ["ilahi" for _ in range(200)]
pop_dataset["sarki_turu"] = ["pop" for _ in range(200)]
rock_dataset["sarki_turu"] = ["rock" for _ in range(200)]
rap_dataset["sarki_turu"] = ["rap" for _ in range(200)]

df_dataset = pd.concat([halk_dataset, sanat_dataset, ilahi_dataset, pop_dataset, rock_dataset, rap_dataset], axis=0, ignore_index=True)

df_dataset = df_dataset.reset_index().rename(columns={'index': 'id'})

# Verilerin Temizlenmesi

In [22]:
def clean_text(text):
    # Satır boşluklarının kaldırılması
    text = text.replace("\n\n", "\n")

    # Satır Sonlarındaki Boşlukların Kaldırılması
    lines = text.split("\n")
    temp_text = ""
    for line in lines:
        temp_text += line.rstrip() + "\n"
    text = temp_text

    # Noktalama İşaretlerinin Kaldırılması
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Küçük Harf Dönüşümü
    text = text.lower()

    return text

df_dataset["sarki_sozu"] = [clean_text(lyrics) for lyrics in df_dataset.loc[:,"sarki_sozu"]]

In [23]:
df_dataset.to_csv("../dataset/dataset.csv", index=False)

##### Stop word'lerin kaldırılması

In [25]:
# Getting stop_words
my_file = open("../raw_data/stop_words.txt", "r")
stop_words = my_file.read()
stop_words = stop_words.replace('\n', ' ').split(" ")
my_file.close()    

def remove_stopwords(text):
    # Split text into lines
    lines = text.split('\n')
    
    # Remove stopwords from each line
    for i, line in enumerate(lines):
        words = line.split()
        filtered_words = [word for word in words if word.lower() not in stop_words]
        lines[i] = ' '.join(filtered_words)
    
    # Join lines back together with line breaks
    return '\n'.join(lines)

df_dataset["sarki_sozu"] = [remove_stopwords(lyrics) for lyrics in df_dataset.loc[:,"sarki_sozu"]]

In [26]:
df_dataset.to_csv("../dataset/dataset_without_stopwords.csv", index=False)

In [ ]:
import pos_tagger

for data in df_dataset:
    pos_tags_list = []
    verb_rate_list = []
    for lyrics in data.loc[:, "sarki_sozu"]:
        pos_tags = pos_tagger.get_pos_tags(lyrics.replace("[CLS]", "").replace("[SEP]", "\n"))
        pos_tags_list.append(pos_tags)
        verb_rate_list.append(round(pos_tags.count(pos_tagger.POSTag.Verb.name) / len(pos_tags), 2))
    data["pos_tagler"] = pos_tags_list
    data["fiil_orani"] = verb_rate_list


In [ ]:
for data in dataset:
    print(sum(data.loc[:, "fiil_orani"]) / len(data.loc[:, "fiil_orani"]))